#試すモデル
*   xgboost(GPU, CPU)
*   lightgbm(CPU)
*   catboost(GPU, CPU)
*   histgradient(CPU)
*   gradient boosting(CPU)
*   (おまけ)TabNet(GPU, CPU)

In [ ]:
#以下のコードを実行したらランタイムを再起動してください
!pip -qqq install lightgbm -U #最新版にアップデート
!pip -qqq install catboost
!pip -qqq install pytorch_tabnet

     |████████████████████████████████| 2.0 MB 5.2 MB/s 
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import time
import os
from IPython.display import display

X, y = fetch_california_housing(return_X_y=True, as_frame=True)

#使うデータ
カリフォルニアの物件価格データ
データ数: 20640
特徴量数: 8

In [ ]:
display(X, y)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


0        4.526
1        3.585
2        3.521
3        3.413
4        3.422
         ...  
20635    0.781
20636    0.771
20637    0.923
20638    0.847
20639    0.894
Name: MedHouseVal, Length: 20640, dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # 訓練データとテストデータに8:2で分ける

In [ ]:
NUM_ROUND = 3000 # 試行回数の設定

#Xgboost

In [ ]:
import xgboost as xgb

xgb_param = {
    'objective': 'reg:squarederror', # 評価指標を２乗誤差に設定
    'tree_method': 'gpu_hist' # GPUを使う設定
}

# データをXgboostフォーマットに変換
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

############### GPU ###############
gpu_res = {} # 結果を保存する辞書
tmp = time.time()
# 訓練開始
print("="*10, "GPU Training", "="*10)
model_gpu = xgb.train(xgb_param, dtrain, NUM_ROUND, evals=[(dtest, 'test')], evals_result=gpu_res, verbose_eval=300)
print("GPU Training Time: %s seconds" % (str(time.time() - tmp)))
#予測出力
pred = model_gpu.predict(xgb.DMatrix(X_test))
gpu_score = mean_squared_error(y_test, pred)
print(f"GPU Score: {gpu_score:.5f}")

############### CPU ###############
tmp = time.time()
xgb_param['tree_method'] = 'hist'
cpu_res = {}
# 訓練開始
print("="*10, "CPU Training", "="*10)
model_cpu = xgb.train(xgb_param, dtrain, NUM_ROUND, evals=[(dtest, 'test')], evals_result=cpu_res, verbose_eval=300)
print("CPU Training Time: %s seconds" % (str(time.time() - tmp)))
#予測出力
pred = model_cpu.predict(xgb.DMatrix(X_test))
cpu_score = mean_squared_error(y_test, pred)
print(f"CPU Score: {cpu_score:.5f}")

========== GPU Training ==========
[0]	test-rmse:1.44646
[300]	test-rmse:0.456367
[600]	test-rmse:0.456911
[900]	test-rmse:0.457586
[1200]	test-rmse:0.458219
[1500]	test-rmse:0.458327
[1800]	test-rmse:0.458408
[2100]	test-rmse:0.458434
[2400]	test-rmse:0.458434
[2700]	test-rmse:0.458434
[2999]	test-rmse:0.458434
GPU Training Time: 24.135740756988525 seconds
GPU Score: 0.21016
========== CPU Training ==========
[0]	test-rmse:1.44479
[300]	test-rmse:0.457656
[600]	test-rmse:0.458125
[900]	test-rmse:0.458779
[1200]	test-rmse:0.459078
[1500]	test-rmse:0.459164
[1800]	test-rmse:0.459236
[2100]	test-rmse:0.459247
[2400]	test-rmse:0.459247
[2700]	test-rmse:0.459247
[2999]	test-rmse:0.459247
CPU Training Time: 12.712014198303223 seconds
CPU Score: 0.21091


#LightGBM


In [ ]:
import lightgbm as lgb

lgb_param = {
    "objective": "regression",
    "metric": "mse",
    "device_type": "cuda"
}

#データをlightGBMフォーマットに変換
dtrain = lgb.Dataset(X_train, label=y_train)
dtest = lgb.Dataset(X_test, label=y_test)

############### CPU ###############
tmp = time.time()
lgb_param['device_type'] = 'cpu'
cpu_res = {}

#callback関数の定義
callbacks = [
    lgb.log_evaluation(300),
    lgb.record_evaluation(cpu_res)
]

# 訓練開始
print("="*10, "CPU Training", "="*10)
model_cpu = lgb.train(lgb_param, dtrain, NUM_ROUND, valid_sets=dtest, callbacks=callbacks)
print(f"CPU Training Time: {(time.time() - tmp):.5f}s seconds")
#予測出力
pred = model_cpu.predict(X_test)
cpu_score = mean_squared_error(y_test, pred)
print(f"CPU Score: {cpu_score:.5f}")

========== CPU Training ==========
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1837
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 2.070980
[300]	valid_0's l2: 0.192368
[600]	valid_0's l2: 0.190575
[900]	valid_0's l2: 0.19101
[1200]	valid_0's l2: 0.191975
[1500]	valid_0's l2: 0.192754
[1800]	valid_0's l2: 0.19331
[2100]	valid_0's l2: 0.193991
[2400]	valid_0's l2: 0.194539
[2700]	valid_0's l2: 0.194823
[3000]	valid_0's l2: 0.195087
CPU Training Time: 4.34246s seconds
CPU Score: 0.19509


#CatBoost

In [ ]:
import catboost as cat

cat_param = {
    "iterations": NUM_ROUND,
    "task_type": "GPU",
    "devices": "0",
    "verbose": 300
}

dtrain = cat.Pool(X_train, label=y_train)
dtest = cat.Pool(X_test, label=y_test)

############### GPU ###############
tmp = time.time()
# 訓練開始
print("="*10, "GPU Training", "="*10)
model_gpu = cat.CatBoostRegressor(**cat_param)
model_gpu.fit(dtrain, eval_set=dtest)
print(f"GPU Training Time: {(time.time() - tmp):.5f}s seconds")
#予測出力
pred = model_gpu.predict(X_test)
gpu_score = mean_squared_error(y_test, pred)
print(f"GPU Score: {gpu_score:.5f}")

############### CPU ###############
tmp = time.time()
cat_param['task_type'] = 'CPU'
# 訓練開始
print("="*10, "CPU Training", "="*10)
model_cpu = cat.CatBoostRegressor(**cat_param)
model_cpu.fit(dtrain, eval_set=dtest)
print(f"CPU Training Time: {(time.time() - tmp):.5f}s seconds")
#予測出力
pred = model_cpu.predict(X_test)
cpu_score = mean_squared_error(y_test, pred)
print(f"CPU Score: {cpu_score:.5f}")

========== GPU Training ==========
Learning rate set to 0.061469
0:	learn: 1.1131698	test: 1.1187512	best: 1.1187512 (0)	total: 22.5ms	remaining: 1m 7s
300:	learn: 0.4532453	test: 0.4708726	best: 0.4708726 (300)	total: 3.59s	remaining: 32.2s
600:	learn: 0.4203955	test: 0.4527602	best: 0.4527602 (600)	total: 6.93s	remaining: 27.7s
900:	learn: 0.4038952	test: 0.4460436	best: 0.4459686 (897)	total: 10.1s	remaining: 23.6s
1200:	learn: 0.3925180	test: 0.4427229	best: 0.4427100 (1191)	total: 13.3s	remaining: 19.9s
1500:	learn: 0.3862254	test: 0.4412647	best: 0.4412643 (1498)	total: 16.2s	remaining: 16.2s
1800:	learn: 0.3807451	test: 0.4399332	best: 0.4398906 (1785)	total: 19.2s	remaining: 12.8s
2100:	learn: 0.3752672	test: 0.4385085	best: 0.4385085 (2100)	total: 22.1s	remaining: 9.45s
2400:	learn: 0.3704167	test: 0.4376982	best: 0.4376982 (2400)	total: 25s	remaining: 6.24s
2700:	learn: 0.3668240	test: 0.4365514	best: 0.4365133 (2667)	total: 28s	remaining: 3.09s
2999:	learn: 0.3645759	test: 0

#HistGradientBoosting

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

hist_param = {
    "loss": "squared_error",
    "max_iter": NUM_ROUND,
    "verbose": 300
}

############### CPU ###############
tmp = time.time()
# 訓練開始
print("="*10, "CPU Training", "="*10)
model_cpu = HistGradientBoostingRegressor(**hist_param)
model_cpu.fit(X_train, y_train)
print(f"CPU Training Time: {(time.time() - tmp):.5f}s seconds")
#予測出力
pred = model_cpu.predict(X_test)
cpu_score = mean_squared_error(y_test, pred)
print(f"CPU Score: {cpu_score:.5f}")

========== CPU Training ==========
Binning 0.001 GB of training data: 0.098 s
Binning 0.000 GB of validation data: 0.020 s
Fitting gradient boosted rounds:
[1/3000] 1 tree, 31 leaves, max depth = 8, train loss: 0.58137, val loss: 0.59511, in 0.096s
[2/3000] 1 tree, 31 leaves, max depth = 8, train loss: 0.51478, val loss: 0.52710, in 0.037s
[3/3000] 1 tree, 31 leaves, max depth = 7, train loss: 0.45868, val loss: 0.47010, in 0.111s
[4/3000] 1 tree, 31 leaves, max depth = 7, train loss: 0.41043, val loss: 0.42066, in 0.011s
[5/3000] 1 tree, 31 leaves, max depth = 9, train loss: 0.36866, val loss: 0.37901, in 0.048s
[6/3000] 1 tree, 31 leaves, max depth = 8, train loss: 0.33490, val loss: 0.34495, in 0.116s
[7/3000] 1 tree, 31 leaves, max depth = 8, train loss: 0.30772, val loss: 0.31755, in 0.010s
[8/3000] 1 tree, 31 leaves, max depth = 8, train loss: 0.28416, val loss: 0.29311, in 0.009s
[9/3000] 1 tree, 31 leaves, max depth = 9, train loss: 0.26369, val loss: 0.27296, in 0.009s
[10/300

#GradientBoosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

hist_param = {
    "loss": "squared_error",
    "n_estimators": 50,
    "verbose": 300
}

############### CPU ###############
tmp = time.time()
# 訓練開始
print("="*10, "CPU Training", "="*10)
model_cpu = GradientBoostingRegressor(**hist_param)
model_cpu.fit(X_train, y_train)
print(f"CPU Training Time: {(time.time() - tmp):.5f}s seconds")
#予測出力
pred = model_cpu.predict(X_test)
cpu_score = mean_squared_error(y_test, pred)
print(f"CPU Score: {cpu_score:.5f}")

========== CPU Training ==========
      Iter       Train Loss   Remaining Time 
         1           1.1940            1.80s
         2           1.0837            1.75s
         3           0.9928            1.69s
         4           0.9123            1.76s
         5           0.8460            1.70s
         6           0.7931            1.65s
         7           0.7442            1.60s
         8           0.7038            1.56s
         9           0.6699            1.52s
        10           0.6412            1.48s
        11           0.6143            1.44s
        12           0.5906            1.40s
        13           0.5703            1.36s
        14           0.5533            1.32s
        15           0.5325            1.29s
        16           0.5188            1.25s
        17           0.5063            1.22s
        18           0.4946            1.19s
        19           0.4831            1.16s
        20           0.4709            1.12s
        21         

#(おまけ)TabNet

In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor

tab_param = {
    "device_name": "cuda",
    "verbose": 3,
}

X_train = X_train.values
X_test = X_test.values
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

############### GPU ###############
tmp = time.time()
# 訓練開始
print("="*10, "GPU Training", "="*10)
model_gpu = TabNetRegressor(**tab_param)
model_gpu.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric=["mse"], max_epochs=NUM_ROUND)
print(f"GPU Training Time: {(time.time() - tmp):.5f}s seconds")
#予測出力
pred = model_gpu.predict(X_test)
gpu_score = mean_squared_error(y_test, pred)
print(f"GPU Score: {gpu_score:.5f}")

############### CPU ###############
tmp = time.time()
# 訓練開始
print("="*10, "CPU Training", "="*10)
tab_param["device_name"] = "cpu"
model_cpu = TabNetRegressor(**tab_param)
model_cpu.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric=["mse"], max_epochs=NUM_ROUND)
print(f"CPU Training Time: {(time.time() - tmp):.5f}s seconds")
#予測出力
pred = model_cpu.predict(X_test)
cpu_score = mean_squared_error(y_test, pred)
print(f"CPU Score: {cpu_score:.5f}")

========== GPU Training ==========
Device used : cuda
epoch 0  | loss: 2.98552 | val_0_mse: 547.89931|  0:00:03s
epoch 3  | loss: 0.55137 | val_0_mse: 2.59755 |  0:00:05s
epoch 6  | loss: 0.45334 | val_0_mse: 114.42751|  0:00:07s
epoch 9  | loss: 0.41024 | val_0_mse: 134.79747|  0:00:09s
epoch 12 | loss: 0.38569 | val_0_mse: 264.38136|  0:00:11s

Early stopping occurred at epoch 14 with best_epoch = 4 and best_val_0_mse = 2.21771
Best weights from best epoch are automatically used!
GPU Training Time: 17.10545s seconds
GPU Score: 2.21771
========== CPU Training ==========
Device used : cpu
epoch 0  | loss: 2.96753 | val_0_mse: 1237.73933|  0:00:00s
epoch 3  | loss: 0.46278 | val_0_mse: 28.89551|  0:00:02s
epoch 6  | loss: 0.40232 | val_0_mse: 70.95249|  0:00:03s
epoch 9  | loss: 0.37545 | val_0_mse: 9.08829 |  0:00:05s
epoch 12 | loss: 0.37093 | val_0_mse: 93.13641|  0:00:06s
epoch 15 | loss: 0.36743 | val_0_mse: 15.99123|  0:00:08s
epoch 18 | loss: 0.35166 | val_0_mse: 6.70646 |  0:00: